# 1. Кофе

Компания, занимающаяся оптовыми поставками зернового кофе, запросила отчёт о продажах по месяцам. Интересуют только те продажи, у которых не было доставки: их количество в месяц, процент плохих отзывов, доходность таких транзакций – в разрезе на продажи с вводом клиентом  промокода и без. 
Имеется выгрузка всех продаж с момента основания компании.
Необходимо написать код на Python, который прочтет выгрузку, проведет необходимые расчеты и запишет таблицу-результат в эксель. 
Поля итоговой таблицы: 
«Месяц» – «Введен промокод» – «Кол-во транзакций» – «Плохих отзывов, %» - «Доходность, %».

**Примечание:**
Доходность можно вычислить по формуле
 Доходность, % = (Оплаченная по транзакциям сумма - Закупочная цена поставленных зерен) / Закупочная цена поставленных зерен * 100, где
Закупочная цена поставленных зерен = сумма Cost всех с нужными характеристиками,
Оплаченная по транзакциям сумма= сумма Paid транзакций с нужными характеристиками.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/transactions.csv', encoding='cp1251', sep='\t')

In [3]:
df.head(3)

,TransactionID,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,0,0,"2605,0",5481,0,0
1,2,2019-Sep-3 11:53:03,90,8307,1,1,"8553,0",5956,1,1
2,3,2019-Oct-15 4:31:02,59,9819,0,1,"9995,0",5170,1,0


In [4]:
df.drop(columns=['AutoIssue'], inplace=True)

In [5]:
df.columns = ['id', 'date', 'count', 'cost', 'delivery', 'bad_review', 'paid', 'weight', 'promo']

In [6]:
df.head(3)

,id,date,count,cost,delivery,bad_review,paid,weight,promo
0,1,2020-Aug-21 8:53:03,67,6325,0,0,"2605,0",5481,0
1,2,2019-Sep-3 11:53:03,90,8307,1,1,"8553,0",5956,1
2,3,2019-Oct-15 4:31:02,59,9819,0,1,"9995,0",5170,1


In [7]:
df.dtypes

id             int64
date          object
count          int64
cost           int64
delivery       int64
bad_review     int64
paid          object
weight         int64
promo          int64
dtype: object

In [8]:
df['paid'] = df['paid'].str.replace(',', '.').astype(float)

In [9]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')

In [10]:
df.head(3)

,id,date,count,cost,delivery,bad_review,paid,weight,promo,month
0,1,2020-08-21 08:53:03,67,6325,0,0,2605.0,5481,0,2020-08
1,2,2019-09-03 11:53:03,90,8307,1,1,8553.0,5956,1,2019-09
2,3,2019-10-15 04:31:02,59,9819,0,1,9995.0,5170,1,2019-10


In [11]:
df['total_cost'] = df['count']  * df['cost']
df['total_paid'] = df['count']  * df['paid']

In [12]:
no_delivery_sales = df.query('delivery == 0') # df[df.delivery == 0]

In [13]:
def t(x):
    return (x > 0).mean() * 100


result = no_delivery_sales.groupby(['month', 'promo']).agg(
    total_count=('count', 'sum'),
    bad_reviews_percentage=('bad_review', t),
    total_cost=('total_cost', 'sum'),
    total_paid=('total_paid', 'sum'),
).reset_index()

In [14]:
result.head(3)

,month,promo,total_count,bad_reviews_percentage,total_cost,total_paid
0,2019-03,0,56,100.000000,480592,215096.0
1,2019-04,0,8932,49.717514,60951285,65637650.0
2,2019-04,1,7115,46.043165,48569250,49360026.0


In [15]:
result['profitability'] = (result['total_paid'] / result['total_cost']) * 100

In [16]:
result.head(3)

,month,promo,total_count,bad_reviews_percentage,total_cost,total_paid,profitability
0,2019-03,0,56,100.000000,480592,215096.0,44.756467
1,2019-04,0,8932,49.717514,60951285,65637650.0,107.688706
2,2019-04,1,7115,46.043165,48569250,49360026.0,101.628141


In [17]:
result.drop(columns=['total_cost', 'total_paid'], inplace=True)

In [18]:
result.head(3)

,month,promo,total_count,bad_reviews_percentage,profitability
0,2019-03,0,56,100.000000,44.756467
1,2019-04,0,8932,49.717514,107.688706
2,2019-04,1,7115,46.043165,101.628141


In [19]:
result.columns = ['Месяц', 'Промокод', 'Кол-во транзакций', 'Плохих отзывов, %', 'Доходность, %']

In [20]:
result.head(3)

,Месяц,Промокод,Кол-во транзакций,"Плохих отзывов, %","Доходность, %"
0,2019-03,0,56,100.000000,44.756467
1,2019-04,0,8932,49.717514,107.688706
2,2019-04,1,7115,46.043165,101.628141


In [21]:
result.to_excel('output/coffe_sales_report.xlsx', index=False)